In [ ]:
import aiida

# must execute it in the first place
aiida.load_profile()


In [ ]:
from aiida.manage.configuration import get_profile

from aiida.common.datastructures import CalcInfo, CodeInfo
from aiida.common.folders import Folder

from aiida.engine import CalcJob, WorkChain
from aiida.engine import calcfunction, workfunction, submit, run, run_get_node

from aiida.plugins import DataFactory, WorkflowFactory

from aiida.parsers.parser import Parser
from aiida.orm import Code
from aiida.orm import load_code, load_node
from aiida.orm import Str, Dict, Float, Int
from aiida.orm import QueryBuilder

import aiida

from ase.io.espresso import write_espresso_in
from ase.io.lammpsdata import read_lammps_data
from ase.build import make_supercell
from ase import Atom, Atoms
import ase

from itertools import combinations_with_replacement
import numpy as np
import os
import subprocess
import shutil
import re
from time import sleep
import spglib
import json

from os.path import expanduser

from tools import NodeBank
from tools import wait_for_node_finished
from alamode_aiida.io import load_atoms
from tools.putstructure import PutStructure


In [ ]:
# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
List = DataFactory('list')


In [ ]:
g_graph = True

with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)
_path


In [ ]:
# codeの定義を行う。
from tools.aiida_support import get_or_create_local_computer, get_or_create_code
from os.path import expanduser
import os

home = expanduser("~")
work_directory = os.path.join(home, 'aiida')
computer_local = get_or_create_local_computer(work_directory, 'localhost')

if True:
    code = get_or_create_code('alamode.alm_suggest',
                              computer_local,
                              '/home/max/Documents/alamode/bin/alm')
    code_alm_sugget = code
else:
    code_alm_sugget = WorkflowFactory('alamode.alm_ALM_suggest')

code_alm_sugget


In [ ]:
# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)


In [ ]:
# 計算するroot directoryの定義
key = 'root_wd'
g_root_wd = nodebank.load(key)


In [ ]:
# harmonic partの計算を行う。
g_prefix = Str("harmonic")
g_norder = Int(1)

g_prefix = nodebank.dump("prefix", g_prefix)
g_norder = nodebank.dump(f"{g_prefix.value}_norder", g_norder)
g_factor = nodebank.load('factor')
g_norder, g_prefix,  g_factor


In [ ]:
g_standardizedstructure = nodebank.load("standardizedstructure")
g_factor = nodebank.load('factor')
g_standardizedstructure.get_ase(), g_factor.get_array('factor')


In [ ]:
# ファイルを置くsubdirectory名の作成
# ディレクトリ名 cwd=formula_supercell_harmonic
# ディレクトリがなければ作成する。
g_almprefix = "_".join([g_standardizedstructure.get_formula(), "x".join(
    map(str, g_factor.get_array("factor").reshape(-1))), g_prefix.value])
g_almprefix = Str(g_almprefix)
print(g_almprefix)

# cwdの設定
g_cwd = Str(os.path.join(g_root_wd.value, g_almprefix.value))
g_cwd = nodebank.load_or_dump("cwd", g_cwd)
# 無ければ作成。
os.makedirs(g_cwd.value, exist_ok=True)


In [ ]:

g_primstructure = nodebank.load("primstructure")
g_superstructure = nodebank.load("superstructure")


## alm suggest

```
Description:

	alm mode="suggest"
	
	default input filename: alm_suggest.in
	default output filename: alm_suggest.out

Inputs:
           code:  required  Code             The `Code` to use for this job.
         norder:  required  Int              1 (harmonic) or 2 (cubic)
      structure:  required  StructureData    structure of cyrstal.
         cutoff:  optional  Dict             distance cutoff
            cwd:  optional  Str              directory where results are saved.
       metadata:  optional                   
           mode:  optional  Str              mode of alm='suggest'
          param:  optional  Dict             optional parameters
         prefix:  optional  Str              string added to the filename.
Outputs:
        pattern:  required  List             pattern of displacement
  remote_folder:  required  RemoteData       Input files necessary to run the process will be stored in this folder node ...
        results:  required  Dict             
      retrieved:  required  FolderData       Files that are retrieved by the daemon will be stored in this node. By defa ...
   remote_stash:  optional  RemoteStashData  Contents of the `stash.source_list` option are stored in this remote folder ...

```

In [ ]:
Str(os.path.join(g_cwd.value, f"{g_prefix.value}_alm_suggest"))  

In [ ]:
codetype =  "calculation"
try:
    if issubclass(code_alm_sugget, WorkChain):
        codetype = "workchain"
except TypeError:
    codetype = "calculation"

if codetype=="workchain":
    g_cutoff_radii = List(list=[[-1]])
    g_nbody=List(list=[2])
    inputs = {'structure': g_superstructure,
             'norder': g_norder,
              'cwd': Str(os.path.join(g_cwd.value, f"{g_prefix.value}_alm_suggest"))  ,
              'cutoff_radii': g_cutoff_radii,
              'nbody': g_nbody
             }
    g_alm_suggest_future = submit(code_alm_sugget, **inputs)
    print(g_alm_suggest_future)
    wait_for_node_finished(g_alm_suggest_future)
else:
    print("code")
    ## inputs
    code = code_alm_sugget
    builder = code.get_builder()
    builder.structure = g_superstructure # equilibrium structure
    builder.norder = g_norder
    builder.cwd = Str(os.path.join(g_cwd.value, f"{g_prefix.value}_alm_suggest"))    
    # calcjobの場合はmetadataを加える。
    builder.metadata = {
        'options': {
            'resources': {'tot_num_mpiprocs': 1, 'num_machines': 1}
        }}

    
    g_alm_suggest_future = nodebank.load_code_or_wait_for_node(f"{g_prefix.value}_alm_suggest", builder)


In [ ]:
g_alm_suggest_future

In [ ]:
## outputs
print(g_alm_suggest_future.outputs.results.attributes)
print(g_alm_suggest_future.outputs.pattern)

In [ ]:
# pattern fileリストはg_alm_suggest_future.outputs.pattern_folderに入る。
# 次の実行ファイルで用いるために保存する。
nodebank.dump(f"{g_prefix.value}_pattern",
              g_alm_suggest_future.outputs.pattern)


In [ ]:
if g_graph:
    !verdi node graph generate $g_alm_suggest_future.pk
